# Google Tasks

In [ ]:
import logging
logging.basicConfig(level = logging.INFO)

from os import environ

from pymongo import MongoClient, errors
from pymongo.collection import Collection

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ["https://www.googleapis.com/auth/tasks.readonly"]

In [ ]:
mongo: Collection = MongoClient(environ["MONGO_URL"])['daily']['tasks_raw']
'id_1' not in mongo.index_information() and mongo.create_index([('id', 1)], unique = True)
# TODO: Stop this line outputting the index name

In [ ]:
def set_credentials():
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            credentials = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(credentials.to_json())
    return credentials

In [ ]:
credentials = set_credentials()
service = build("tasks", "v1", credentials = credentials)

In [ ]:
tasklists = service.tasklists().list().execute().get("items", [])
logging.info(f'Logging {len(tasklists)} tasklist(s)...')

for tasklist in tasklists:
    request = service.tasks().list(tasklist = tasklist['id'])
    while request:
        tasks_doc = request.execute()
        tasks_list = tasks_doc.get("items", [])
        logging.info(f'Logging {len(tasks_list)} tasks from page <page_number> of {tasklist["title"]}...')
        try:
            mongo.insert_many(tasks_list)
        except errors.BulkWriteError:
            # TODO: This assumes an ID will never move up or down the tasklist, which is not always true
            logging.warning("Duplicate task(s) detected, skipping...")
            break
        request = service.tasks().list_next(request, tasks_doc)


In [ ]:
%pip install pyexcel-ods
import pandas as pd

In [ ]:
data = list(mongo.find())

df = pd.DataFrame(data)

df.to_excel('tasks.ods', engine = 'odf', index = False)

In [ ]:
df.to_csv('tasks.csv', index = False)